In [68]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import category_encoders as ce
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from xgboost import train_values_copyGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer


train_values = pd.read_csv('train_values.csv', sep=',')
train_labels = pd.read_csv('train_labels.csv', sep=',')
x_test = pd.read_csv('test_values.csv', sep=',')

train_values_copy = train_values.copy()
train_values_copy.drop('building_id', inplace=True, axis=1)
y = train_labels.copy()
y.drop('building_id', inplace=True, axis=1)
x_test_dropped = x_test.copy()
x_test_dropped.drop('building_id', inplace=True, axis=1)


     |████████████████████████████████| 1.7 MB 2.3 MB/s eta 0:00:01


In [69]:
train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]] = train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]].astype(str)
x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]] = x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]].astype(str)


In [70]:
#generate the encoded columns
first_ce = ce.OneHotEncoder(cols=["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","legal_ownership_status"]) 
ce_te = ce.TargetEncoder(cols=["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"])
xx = train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]]
x_test_target_encoded = x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]]
yy = y['damage_grade']

#fit and transform the target encoded values
ce_te.fit(xx,yy)
train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]] = ce_te.transform(xx)
x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]] = ce_te.transform(x_test_target_encoded)

#fit and transform the one hot encoded values
train_values_copy = first_ce.fit_transform(train_values_copy)
x_test_dropped = first_ce.fit_transform(x_test_dropped)

In [ ]:
#Do a linear regression to check p-values
reg = LinearRegression().fit(train_values_copy, y)
    
est = sm.OLS(y, train_values_copy)
est2 = est.fit()
est2.summary()

In [63]:
dropped_columns_encoded_x_train = train_values_copy.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])
dropped_columns_encoded_x_test = x_test_dropped.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])

In [74]:
from sklearn.ensemble import RandomForestClassifier

reg = LinearRegression().fit(dropped_columns_encoded_x_train, y)
    
est = sm.OLS(y, dropped_columns_encoded_x_train)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           damage_grade   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     4416.
Date:                Fri, 29 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:02:14   Log-Likelihood:            -1.5939e+05
No. Observations:              260601   AIC:                         3.189e+05
Df Residuals:                  260548   BIC:                         3.194e+05
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
geo_level_1_id                             0.0384      0.005      7.153      0.000       0.028       0.049
geo_level_2_id                            -0.0336      0.007     -5.152      0.000      -0.046      -0.021
geo_level_3_id                             0.9114      0.005    179.384      0.000       0.901       0.921
count_floors_pre_eq                        0.0012      0.002      0.527      0.598      -0.003       0.006
age                                        0.0001    1.2e-05      9.918      0.000    9.56e-05       0.000
area_percentage                           -0.0002      0.000     -0.928      0.354      -0.001       0.000
height_percentage                          0.0026      0.001      3.441      0.001       0.001       0.004
land_surface_condition_1                3.483e+08   1.27e+09      0.275      0.784   -2.14e+09    2.83e+09
land_surface_condition_2                3.483e+08   1.27e+09      0.275      0.784   -2.14e+09    2.83e+09
land_surface_condition_3                3.483e+08   1.27e+09      0.275      0.784   -2.14e+09    2.83e+09
foundation_type_1                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_2                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_3                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_4                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_5                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
roof_type_1                             3.099e+07   1.13e+08      0.275      0.784    -1.9e+08    2.52e+08
roof_type_2                             3.099e+07   1.13e+08      0.275      0.784    -1.9e+08    2.52e+08
roof_type_3                             3.099e+07   1.13e+08      0.275      0.784    -1.9e+08    2.52e+08
ground_floor_type_1                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_2                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_3                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_4                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_5                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
other_floor_type_1                     -7.529e+07   2.74e+08     -0.275      0.784   -6.13e+08    4.62e+08
other_floor_type_2                     -7.529e+07   2.74e+08     -0.275      0.784   -6.13e+08    4.62e+08
other_floor_type_3                     -7.529e+07   2.74e+08     -0.275      0.784   -6

In [64]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(min_samples_split=20, n_estimators=200)
clf = clf.fit(dropped_columns_encoded_x_train,y)
y_pred=clf.predict(dropped_columns_encoded_x_test)
y_pred

array([3, 2, 3, ..., 2, 2, 2])

In [10]:
from sklearn.model_selection import cross_val_score
y_train_vector = y.values.flatten()
print(np.mean(cross_val_score(clf, dropped_columns_encoded_x_train, y_train_vector)))

0.7408145070896852


In [59]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2, random_state=0).fit(dropped_columns_encoded_x_train, y)
ypred_gradient_boosting = clf.predict(dropped_columns_encoded_x_test)
ypred_gradient_boosting


array([3, 2, 3, ..., 2, 2, 2])

In [55]:
estimator = train_values_copyGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)
parameters = {
    'max_depth': [6, 10],
    'learning_rate': [0.1, 0.2]
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    verbose=True,
    error_score='raise'
)
grid_search.fit(dropped_columns_encoded_x_train, y.values.flatten() - 1)

sorted(grid_search.cv_results_.keys())
grid_search.best_estimator_


Fitting 5 folds for each of 4 candidates, totalling 20 fits


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42, ...)

In [73]:
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
xgbc = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              monotone_constraints='()', n_estimators=100,
              n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42,)
print(xgbc)
xgbc.fit(dropped_columns_encoded_x_train, y-1)
ypred_xgboost = xgbc.predict(dropped_columns_encoded_x_test) + 1

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42, ...)


In [31]:
ypred_xgboost

array([3, 2, 3, ..., 2, 2, 2])

In [65]:
from pathlib import Path  
submission_random_forest = x_test["building_id"]
i = pd.DataFrame(np.array(y_pred))

submission_concatenated = pd.concat([submission_random_forest, i], ignore_index="true", axis=1)
submission_renamed_columns = submission_concatenated.rename(columns={0: "building_id", 1: "damage_grade"})
filepath = Path('./submission_richter_prediction.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
submission_renamed_columns.to_csv(filepath, index=False)